In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:5]:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/plant-seedlings-classification/sample_submission.csv
/kaggle/input/plant-seedlings-classification/test/fd87b36ae.png
/kaggle/input/plant-seedlings-classification/test/0e8492cb1.png
/kaggle/input/plant-seedlings-classification/test/8d6acbe9b.png
/kaggle/input/plant-seedlings-classification/test/54b3afd58.png
/kaggle/input/plant-seedlings-classification/test/6049234e6.png
/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/4ae939d7d.png
/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/b8664f705.png
/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/628b08c82.png
/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/9ab3b61db.png
/kaggle/input/plant-seedlings-classification/train/Scentless Mayweed/086894274.png
/kaggle/input/plant-seedlings-classification/train/Common wheat/df584ca28.png
/kaggle/input/plant-seedlings-classification/train/Common wheat/9026da493.png
/kaggle/input/plant-seedlings-classificati

In [6]:
# glob.glob() : 파일들의 경로를 리스트로 반환
import glob
train = pd.DataFrame({'path':glob.glob('/kaggle/input/plant-seedlings-classification/train/*/*')})
train['label'] = train['path'].apply(lambda x : x.split('/')[-2])
train

,path,label
0,/kaggle/input/plant-seedlings-classification/t...,Scentless Mayweed
1,/kaggle/input/plant-seedlings-classification/t...,Scentless Mayweed
2,/kaggle/input/plant-seedlings-classification/t...,Scentless Mayweed
3,/kaggle/input/plant-seedlings-classification/t...,Scentless Mayweed
4,/kaggle/input/plant-seedlings-classification/t...,Scentless Mayweed
...,...,...
4745,/kaggle/input/plant-seedlings-classification/t...,Shepherds Purse
4746,/kaggle/input/plant-seedlings-classification/t...,Shepherds Purse
4747,/kaggle/input/plant-seedlings-classification/t...,Shepherds Purse
4748,/kaggle/input/plant-seedlings-classification/t...,Shepherds Purse


### 이미지 사이즈 미리 줄이기 &rarr; 교차검증 학습할때 소요시간 &darr;


In [8]:
import cv2
from tqdm import tqdm
# image = cv2.resize(image, dsize=(256,256))

os.makedirs('train_256', exist_ok=True)

for path in tqdm(train['path']):
    image = cv2.imread(path)
    image = cv2.resize(image, dsize=(256,256))
    
    name = 'train_256/' + path.split('/')[-1]
    cv2.imwrite(name, image)

100%|██████████| 4750/4750 [01:51<00:00, 42.44it/s]


In [9]:
train['path'] = "train_256/" + train['path'].apply(lambda x: x.split('/')[-1])
train.head()

,path,label
0,train_256/4ae939d7d.png,Scentless Mayweed
1,train_256/b8664f705.png,Scentless Mayweed
2,train_256/628b08c82.png,Scentless Mayweed
3,train_256/9ab3b61db.png,Scentless Mayweed
4,train_256/086894274.png,Scentless Mayweed


In [29]:
train['label'].value_counts()

Loose Silky-bent             654
Common Chickweed             611
Scentless Mayweed            516
Small-flowered Cranesbill    496
Fat Hen                      475
Charlock                     390
Sugar beet                   385
Cleavers                     287
Black-grass                  263
Shepherds Purse              231
Common wheat                 221
Maize                        221
Name: label, dtype: int64

In [10]:
# from sklearn.model_selection import train_test_split
# from tensorflow.keras import *
# from tensorflow.keras.layers import *
# from tensorflow.keras.applications.efficientnet import EfficientNetB1
# from keras.preprocessing.image import ImageDataGenerator

# X_train, X_valid = train_test_split(train,test_size=0.2, random_state=42,stratify=train['label'])

# idg = ImageDataGenerator()

# train_generator = idg.flow_from_dataframe(X_train, x_col = 'path',y_col = 'label',batch_size=32)
# valid_generator = idg.flow_from_dataframe(X_valid, x_col = 'path',y_col = 'label',batch_size=32)

Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.


In [14]:
test = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')
test['path'] = "/kaggle/input/plant-seedlings-classification/test/" + test['file']

os.makedirs('test_256', exist_ok=True)

for path in tqdm(test['path']):
    image = cv2.imread(path)
    image = cv2.resize(image, dsize=(256,256))
    
    name = 'test_256/' + path.split('/')[-1]
    cv2.imwrite(name, image)
    
test['path'] = 'test_256/' + test['file']
test_generator = idg.flow_from_dataframe(test, x_col = 'path' ,y_col = None,batch_size=32, class_mode=None, shuffle=False)

100%|██████████| 794/794 [00:06<00:00, 125.34it/s]

Found 794 validated image filenames.


## Cross Validation

In [15]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.applications.efficientnet import EfficientNetB1
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import *

skf = StratifiedKFold(n_splits=5, random_state=2023, shuffle=True)
result = 0


for train_index, valid_index in skf.split(train,train['label']):
    X_train = train.iloc[train_index]
    X_valid = train.iloc[valid_index]
    
    idg = ImageDataGenerator()
    train_generator = idg.flow_from_dataframe(X_train, x_col = 'path',y_col = 'label',batch_size=32)
    valid_generator = idg.flow_from_dataframe(X_valid, x_col = 'path',y_col = 'label',batch_size=32)
    
    model = Sequential()
    model.add(EfficientNetB1(include_top=False, weights = 'imagenet',pooling = 'avg'))
    model.add(Dense(12,activation='softmax'))
    
    es = EarlyStopping(patience=3,restore_best_weights=True)
    rl = ReduceLROnPlateau(patience = 2,verbose=1)
    
    model.compile(metrics=['acc'],optimizer='adam', loss='categorical_crossentropy')
    model.fit(train_generator, validation_data = valid_generator, epochs=5, callbacks=[es,rl])  # , steps_per_epoch=1 : 에폭당 1iter 만 수행 / 코드 돌아가는지 확인할 때 활
    
    result += model.predict(test_generator) / 5

Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Epoch 1/5


2023-04-18 15:21:55.167003: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_2/efficientnetb1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


119/119 [==============================] - 103s 430ms/step - loss: 0.5943 - acc: 0.8000 - val_loss: 1.0380 - val_acc: 0.7316 - lr: 0.0010
Epoch 2/5
119/119 [==============================] - 49s 412ms/step - loss: 0.2076 - acc: 0.9321 - val_loss: 0.3605 - val_acc: 0.8916 - lr: 0.0010
Epoch 3/5
119/119 [==============================] - 49s 410ms/step - loss: 0.1159 - acc: 0.9624 - val_loss: 0.2658 - val_acc: 0.9347 - lr: 0.0010
Epoch 4/5
119/119 [==============================] - 49s 408ms/step - loss: 0.0997 - acc: 0.9658 - val_loss: 0.4414 - val_acc: 0.8768 - lr: 0.0010
Epoch 5/5
25/25 [==============================] - 5s 116ms/step
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Epoch 1/5


2023-04-18 15:27:36.775804: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_3/efficientnetb1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


119/119 [==============================] - 104s 432ms/step - loss: 0.5990 - acc: 0.8008 - val_loss: 0.7558 - val_acc: 0.7632 - lr: 0.0010
Epoch 2/5
119/119 [==============================] - 49s 415ms/step - loss: 0.2131 - acc: 0.9258 - val_loss: 0.3163 - val_acc: 0.8968 - lr: 0.0010
Epoch 3/5
119/119 [==============================] - 49s 411ms/step - loss: 0.1258 - acc: 0.9550 - val_loss: 0.2298 - val_acc: 0.9200 - lr: 0.0010
Epoch 4/5
119/119 [==============================] - 50s 417ms/step - loss: 0.0947 - acc: 0.9668 - val_loss: 0.2146 - val_acc: 0.9189 - lr: 0.0010
Epoch 5/5
25/25 [==============================] - 5s 107ms/step
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Epoch 1/5


...

119/119 [==============================] - 103s 432ms/step - loss: 0.6110 - acc: 0.8039 - val_loss: 0.8572 - val_acc: 0.7758 - lr: 0.0010
Epoch 2/5
119/119 [==============================] - 50s 416ms/step - loss: 0.2002 - acc: 0.9329 - val_loss: 0.7334 - val_acc: 0.8316 - lr: 0.0010
Epoch 3/5
119/119 [==============================] - 49s 412ms/step - loss: 0.1309 - acc: 0.9576 - val_loss: 0.4190 - val_acc: 0.8821 - lr: 0.0010
Epoch 4/5
119/119 [==============================] - 49s 411ms/step - loss: 0.1129 - acc: 0.9626 - val_loss: 0.4335 - val_acc: 0.8779 - lr: 0.0010
Epoch 5/5
25/25 [==============================] - 5s 114ms/step
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Epoch 1/5


2023-04-18 15:38:02.584272: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_5/efficientnetb1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


119/119 [==============================] - 104s 437ms/step - loss: 0.5804 - acc: 0.8045 - val_loss: 0.6800 - val_acc: 0.7947 - lr: 0.0010
Epoch 2/5
119/119 [==============================] - 49s 413ms/step - loss: 0.1750 - acc: 0.9397 - val_loss: 0.4925 - val_acc: 0.8579 - lr: 0.0010
Epoch 3/5
119/119 [==============================] - 49s 413ms/step - loss: 0.1358 - acc: 0.9568 - val_loss: 0.2702 - val_acc: 0.9137 - lr: 0.0010
Epoch 4/5
119/119 [==============================] - 49s 412ms/step - loss: 0.1005 - acc: 0.9637 - val_loss: 0.2981 - val_acc: 0.9295 - lr: 0.0010
Epoch 5/5
119/119 [==============================] - ETA: 0s - loss: 0.0742 - acc: 0.9755
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
25/25 [==============================] - 5s 115ms/step
Found 3800 validated image filenames belonging to 12 classes.
Found 950 validated image filenames belonging to 12 classes.
Epoch 1/5


2023-04-18 15:43:13.155044: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_6/efficientnetb1/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


119/119 [==============================] - 104s 429ms/step - loss: 0.6347 - acc: 0.7889 - val_loss: 0.5974 - val_acc: 0.8147 - lr: 0.0010
Epoch 2/5
119/119 [==============================] - 48s 406ms/step - loss: 0.2020 - acc: 0.9324 - val_loss: 0.2267 - val_acc: 0.9274 - lr: 0.0010
Epoch 3/5
119/119 [==============================] - 48s 407ms/step - loss: 0.1276 - acc: 0.9582 - val_loss: 0.6337 - val_acc: 0.8905 - lr: 0.0010
Epoch 4/5
119/119 [==============================] - ETA: 0s - loss: 0.1275 - acc: 0.9621
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
119/119 [==============================] - 48s 406ms/step - loss: 0.1275 - acc: 0.9621 - val_loss: 0.2312 - val_acc: 0.9337 - lr: 0.0010
Epoch 5/5
25/25 [==============================] - 5s 112ms/step


## 정답 레이블 매핑 후 제출

In [32]:
class_name = X_train.label.tolist()
class_num = train_generator.classes
dic = {key:val for key,val in zip(class_num,class_name)}


In [37]:
sub = pd.read_csv('/kaggle/input/plant-seedlings-classification/sample_submission.csv')
result_class = np.argmax(result,axis=1)
result = np.vectorize(dic.get)(result_class)
sub['species'] = result
sub.to_csv('sub.csv',index=False)

array(['Small-flowered Cranesbill', 'Fat Hen', 'Sugar beet',
       'Common Chickweed', 'Sugar beet', 'Loose Silky-bent',
       'Common Chickweed', 'Fat Hen', 'Loose Silky-bent',
       'Loose Silky-bent', 'Fat Hen', 'Small-flowered Cranesbill',
       'Sugar beet', 'Scentless Mayweed', 'Sugar beet', 'Fat Hen',
       'Scentless Mayweed', 'Scentless Mayweed', 'Common Chickweed',
       'Shepherds Purse', 'Common Chickweed', 'Small-flowered Cranesbill',
       'Shepherds Purse', 'Sugar beet', 'Sugar beet', 'Maize',
       'Scentless Mayweed', 'Scentless Mayweed', 'Common Chickweed',
       'Sugar beet', 'Common Chickweed', 'Shepherds Purse', 'Black-grass',
       'Sugar beet', 'Loose Silky-bent', 'Black-grass',
       'Loose Silky-bent', 'Loose Silky-bent', 'Common Chickweed',
       'Maize', 'Loose Silky-bent', 'Common wheat', 'Common Chickweed',
       'Small-flowered Cranesbill', 'Sugar beet', 'Sugar beet', 'Maize',
       'Scentless Mayweed', 'Maize', 'Small-flowered Cranesbill',
 